In [1]:
import sys

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    sys.path.append('/content/drive/MyDrive/ApplAI/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

Usando dispositivo: cpu


In [3]:
%pip install -y pytorch torchvision torchaudio cpuonly -c pytorch
%pip install sentence-transformers
%pip install skorch


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -y


In [4]:
import pandas as pd

base_path = '/content/drive/MyDrive/ApplAI/'  # adaptá este nombre

# Elegí el que quiera usar
modelUse = "params/model2.py"

modelUsed = base_path+modelUse
df = pd.read_csv(base_path+"dataset/plain_text_resume_data.csv")

In [5]:
from funcs_src import *

config = extract_config_from_file(modelUsed)
param_grid = config["param_grid"]
CV = config["cv"]

In [ ]:
from sentence_transformers import SentenceTransformer
from huggingface_hub import login
import os

# Esto descarga y guarda el modelo localmente
model_name = config["model"]
local_path = f'./{model_name}'

if os.path.isdir(local_path):
    # Ya existe: carga desde disco
    model = SentenceTransformer(local_path)
else:
    # No existe: descarga y guarda
    login(token="")
    model = SentenceTransformer(model_name)
    model.save(local_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split
from funcs_src import *

In [8]:
train_df, test_df = train_test_split(
    df,
    train_size=0.8,
    test_size=0.2,
    random_state=42,
)

XTrain, YTrain = getDatasetEmbeddings(model, train_df, config["typeEmbedding"])
XTest, YTest = getDatasetEmbeddings(model, test_df, config["typeEmbedding"])

KeyboardInterrupt: 

In [ ]:
import torch
from skorch import NeuralNetRegressor

net = NeuralNetRegressor(
    module=STClassifier,
    module__model_name=config["model"],
    module__model_downloaded=True,
    module__device='cuda' if torch.cuda.is_available() else 'cpu',
    max_epochs=5,
    lr=1e-3,
    iterator_train__shuffle=True,
    train_split=None,
    verbose=1
)

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np

# RMSE que utilizaremos
rmse_scorer = make_scorer(
    lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
    greater_is_better=False
)

gs = GridSearchCV(
    estimator=net,
    param_grid=param_grid,
    scoring=rmse_scorer,
    cv=3,
    verbose=3,
    n_jobs=1,
    error_score='raise'
)

In [ ]:
gs.fit(XTrain, YTrain)

In [ ]:
best_rmse = -gs.best_score_
print("Mejor RMSE:", best_rmse)

In [ ]:
best_model = gs.best_estimator_

# Hacemos predicciones con el set de test
y_pred = best_model.predict(XTest)

rmse_test = np.sqrt(mean_squared_error(YTest, y_pred))

print(f"RMSE en test: {rmse_test:.4f}")

In [ ]:
save_final_model(
    gs,
    config_py_path=modelUsed,
    base_name="finalmodel",
    out_root="modelsSave"
)